In [1]:
TOTAL_CLIENTS = 3  # number of participants in the lab
CLIENT_ID = 1  # between 0 and TOTAL_CLIENTS-1

In [2]:
import torch
import device_data

training_dataset = device_data.get_client_training_data(CLIENT_ID, TOTAL_CLIENTS)
train_loader = torch.utils.data.DataLoader(training_dataset, batch_size=16)
test_dataset = device_data.get_test_data()
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
import clientlib
SERVER = 'federated-learning.in8.itec.kit.edu:80'

# this reaches 76% within 10 rounds with 4 clients

torch.set_num_threads(1)

import time
from tqdm import tqdm
import sys

device = 'cuda'

def train(model, optimizer, criterion, trainloader, device='cpu'):
    model.train()
    model.to(device)

    for _, (inputs, targets) in enumerate(tqdm(trainloader, ncols=80,
                                               file=sys.stdout, desc="Training", leave=False)):
        
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

def test(model, testloader, device='cpu'):
    num_correct = 0
    num_samples = 0
    
    model.eval()

    for _, (inputs, targets) in enumerate(tqdm(testloader, ncols=80,
                                               file=sys.stdout, desc="Testing", leave=False)):
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        predicted = torch.argmax(outputs, dim=1) 
        num_correct += (predicted == targets).sum().item()
        num_samples += predicted.size(0)

    return num_correct/num_samples

last_trained_round = None

while True:
    print("Starting round")
    
    if ( last_trained_round != round or last_trained_round != None) :
        clientlib.wait_for_next_round(SERVER, last_trained_round=last_trained_round, join_late_by_max=10)
    
    model, model_metadata = clientlib.get_model_and_notify_client_started(SERVER, CLIENT_ID)
    optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    criterion = torch.nn.CrossEntropyLoss()

    print(f'downloaded model -> train (round {model_metadata["round"]})')

    start = time.time()
    train(model, optimizer,criterion, train_loader, device=device)
    end = time.time()
    print(f'training took {end-start:.1f}s')

    print('upload updated model')

    clientlib.upload_updated_model(SERVER, CLIENT_ID, model, model_metadata)

    last_trained_round = model_metadata['round']
    
    #we measure the accuracy after 10 rounds
    if model_metadata['round'] % 2 == 0:
        accuracy = test(model, test_loader, device=device)
        print(f'Accuracy: ({accuracy:.2f})')
        break

Starting round
downloaded model -> train (round 5)


KeyboardInterrupt: 